# **✨State-Value $V^\pi(s)$ & Action-Value Functions $Q^\pi(s, a)$**

## **📑Table of Contents**

1. [Policies $π$ and State-Value Functions $V^π(s)$](#1-policies-π-and-state-value-functions-vπs)
   - 1.1 [Policy](#11-policy)
   - 1.2 [Grid World Policy Example](#12-grid-world-policy-example)
   - 1.3 [State-Value Functions $V^π(s)$](#13-state-value-functions-vπs)
   - 1.4 [Computing State-Values with Recursion](#14-computing-state-values-with-recursion)

2. [The Bellman Equation](#2-the-bellman-equation)
   - 2.1 [Bellman Equation (Recursive Form)](#21-bellman-equation-recursive-form)
   - 2.2 [Mathematical Foundation of the Bellman Equation](#22-mathematical-foundation-of-the-bellman-equation)
   - 2.3 [Intuitive Understanding](#23-intuitive-understanding)
   - 2.4 [Bellman Equation Implementation](#24-bellman-equation-implementation)
   - 2.5 [Bellman Optimality Equation](#25-bellman-optimality-equation)
   - 2.6 [Properties and Convergence](#26-properties-and-convergence)

3. [Action-Value Functions $Q^π(s, a)$](#3-action-value-functions-qπs)
   - 3.1 [What $Q^π(s,a)$ Means](#31-what-qπsa-means)
   - 3.2 [Break the Return into Immediate + Future (Derivation)](#32-break-the-return-into-immediate--future-derivation)
   - 3.3 [Deterministic One-step Bellman Form](#33-deterministic-one-step-bellman-form)
   - 3.4 [Stochastic Transitions — Expectation over Next States](#34-stochastic-transitions--expectation-over-next-states)
   - 3.5 [Bellman Expectation in Terms of $Q$ (No $V$ Needed)](#35-bellman-expectation-in-terms-of-q-no-v-needed)
   - 3.6 [Relationship $V^π ↔ Q^π$](#36-relationship-vπ--qπ)
   - 3.7 [Bellman Optimality Equation (Why the Max Appears)](#37-bellman-optimality-equation-why-the-max-appears)
   - 3.8 [Sample-based Updates (How Algorithms Use These Formulas)](#38-sample-based-updates-how-algorithms-use-these-formulas)
   - 3.9 [Terminal-state Conventions](#39-terminal-state-conventions)
   - 3.10 [Quick Intuitive Checklist](#310-quick-intuitive-checklist)
   - 3.11 [Quick Comparison: $V^π$ vs $Q^π$](#311-quick-comparison-vπ-vs-qπ)


## **🔖1. Policies $\pi$ and State-Value Functions $V^\pi(s)$**

### 1.1 **Policy** 

> Policies: A `policy` $\pi$  is a strategy that defines how an `agent` selects `actions` based on its `current state` to maximize expected `rewards`.

#### Mathematical Foundation
- **Definition**: Policy $\pi$ are the core decision-making mechanism that maps environmental states to specific actions.

#### Policy Types:
- **Deterministic Policy**: $\pi(s) = a$
    - Maps each state to exactly one action
    - Simpler to analyze and implement
    - Sufficient for optimal policies in MDPs

  - **Stochastic Policy**: $\pi(a|s) = P(A_t = a | S_t = s)$
    - Probability distribution over actions for each state
    - More flexible for exploration
    - Required for some advanced algorithms

#### Mathematical Properties:
- For stochastic policies: $\sum_a \pi(a|s) = 1$ for all states $s$
    - Where:
        - $\pi(a|s)$ = Probability of selecting action $a$ in state $s$
        - Sum over all actions must equal 1 (valid probability distribution)

### 1.2 **Grid World Policy Example**

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

print("=== CUSTOM 3x3 GRIDWORLD WITH POLICY AND STATE-VALUES ===")
print()

# -------------------------------
# 1. Custom 3x3 GridWorld Env
# -------------------------------
class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["ansi"]}
    
    def __init__(self):
        super().__init__()
        self.shape = (3, 3)
        self.observation_space = spaces.Discrete(9)
        self.action_space = spaces.Discrete(4)  # 0:left,1:down,2:right,3:up
        
        # Define rewards
        self.terminal_state = 8
        self.rewards = {8: 10, 4: -2, 7: -2}
        
        # Precompute P like Gym
        self.P = {s: {a: [] for a in range(4)} for s in range(9)}
        for s in range(9):
            for a in range(4):
                ns = self._move(s, a)
                r = self.rewards.get(ns, -1)
                done = ns == self.terminal_state
                self.P[s][a] = [(1.0, ns, r, done)]  # deterministic
                
    def _move(self, state, action):
        if state == self.terminal_state:
            return state
        row, col = state // 3, state % 3
        if action == 0:    # left
            col = max(0, col - 1)
        elif action == 1:  # down
            row = min(2, row + 1)
        elif action == 2:  # right
            col = min(2, col + 1)
        elif action == 3:  # up
            row = max(0, row - 1)
        return row * 3 + col
    
    def render(self, mode="ansi"):
        grid = np.full(self.shape, " ")
        grid[0,0] = "S"  # start
        grid[2,2] = "D"  # diamond
        grid[1,1] = grid[1,2] = "M"  # mountains
        return "\n".join([" ".join(row) for row in grid])

# Create environment
env = GridWorldEnv()
num_states = env.observation_space.n
gamma = 1

# -------------------------------
# 2. Define a deterministic policy
# -------------------------------
policy = {
    0: 1,  # down
    1: 2,  # right
    2: 1,  # down
    3: 1,  # down
    4: 3,  # up
    5: 1,  # down
    6: 2,  # right
    7: 3,  # up
    8: 0   # terminal
}

# -------------------------------
# 3. Compute state values
# -------------------------------
def compute_state_value(state):
    if state == env.terminal_state:
        return 0
    
    action = policy[state]
    _, next_state, reward, _ = env.P[state][action][0]
    return reward + gamma * compute_state_value(next_state)

V = {s: compute_state_value(s) for s in range(num_states)}

# -------------------------------
# 4. Display results
# -------------------------------
print("Custom 3x3 GridWorld Layout:")
print(env.render())
print()
print("State-values:", V)

=== CUSTOM 3x3 GRIDWORLD WITH POLICY AND STATE-VALUES ===

Custom 3x3 GridWorld Layout:
S    
  M M
    D

State-values: {0: 1, 1: 8, 2: 9, 3: 2, 4: 7, 5: 10, 6: 3, 7: 5, 8: 0}


In [ ]:
import numpy as np

# ==========================
# 1. POLICIES
# ==========================

# Actions: 0: left, 1: down, 2: right, 3: up
policy1 = {
    0: 1,  # down
    1: 2,  # right
    2: 1,  # down
    3: 1,  # down
    4: 3,  # up
    5: 1,  # down
    6: 2,  # right
    7: 3   # up
}

policy2 = {
    0: 2,  # right
    1: 2,  # right
    2: 1,  # down
    3: 2,  # right
    4: 2,  # right
    5: 1,  # down
    6: 2,  # right
    7: 2   # right
}

action_names = {0: 'left', 1: 'down', 2: 'right', 3: 'up'}

print("Policy 1:", {s: action_names[a] for s, a in policy1.items()})
print("Policy 2:", {s: action_names[a] for s, a in policy2.items()})
print()

# ==========================
# 2. ENVIRONMENT MODEL (P)
# ==========================
gamma = 1.0          # Discount factor
num_states = 9       # 3x3 grid
terminal_state = 8   # Diamond

# Build transition table: P[state][action] = [(prob, next_state, reward, done)]
P = {s: {a: [] for a in range(4)} for s in range(num_states)}

def move(state, action):
    """Return next state after taking an action."""
    if state == terminal_state:
        return state
    
    row, col = state // 3, state % 3
    if action == 0:    # left
        col = max(0, col - 1)
    elif action == 1:  # down
        row = min(2, row + 1)
    elif action == 2:  # right
        col = min(2, col + 1)
    elif action == 3:  # up
        row = max(0, row - 1)
    return row * 3 + col

def reward(next_state):
    """Return reward for landing in next_state."""
    if next_state == 8:   # Diamond
        return 10
    elif next_state in [4, 7]:  # Mountains
        return -2
    else:  # All other states
        return -1

# Fill transition table
for s in range(num_states):
    for a in range(4):
        ns = move(s, a)
        r = reward(ns)
        done = (ns == terminal_state)
        P[s][a] = [(1.0, ns, r, done)]   # deterministic env

# ==========================
# 3. STATE-VALUE FUNCTIONS
# ==========================
print("2. STATE-VALUE FUNCTIONS")
print("========================")
print("V(s) = Expected return starting from state s following policy π")
print()

def compute_state_value(state, policy):
    """Bellman expectation with env.P"""
    if state == terminal_state:
        return 0
    
    action = policy[state]
    transitions = P[state][action]
    
    value = 0
    for prob, next_state, reward, _ in transitions:
        value += prob * (reward + gamma * compute_state_value(next_state, policy))
    return value

# Calculate state values for both policies
print("Computing state values...")
print()

V1 = {s: compute_state_value(s, policy1) for s in range(num_states)}
V2 = {s: compute_state_value(s, policy2) for s in range(num_states)}

# ==========================
# 4. RESULTS
# ==========================
print("RESULTS:")
print("========")
print("State-values for Policy 1:", V1)
print("State-values for Policy 2:", V2)
print()

# Example calculation walkthrough
print("EXAMPLE CALCULATION (Policy 1, State 2):")
print("========================================")
state = 2
action = policy1[state]
prob, next_state, reward, _ = P[state][action][0]
print(f"State 2 → Action {action_names[action]} → State {next_state}")
print(f"Reward: {reward}")
print(f"V(2) = {reward} + {gamma} × V({next_state}) = {reward} + {gamma} × {V1[next_state]} = {V1[2]}")
print()

# Compare policies
print("POLICY COMPARISON:")
print("==================")
total1 = sum(V1[s] for s in range(8))  # exclude terminal
total2 = sum(V2[s] for s in range(8))
print(f"Total value (Policy 1): {total1}")
print(f"Total value (Policy 2): {total2}")
print(f"Better policy: Policy {'2' if total2 > total1 else '1'}")

Policy 1: {0: 'down', 1: 'right', 2: 'down', 3: 'down', 4: 'up', 5: 'down', 6: 'right', 7: 'up'}
Policy 2: {0: 'right', 1: 'right', 2: 'down', 3: 'right', 4: 'right', 5: 'down', 6: 'right', 7: 'right'}

2. STATE-VALUE FUNCTIONS
V(s) = Expected return starting from state s following policy π

Computing state values...

RESULTS:
State-values for Policy 1: {0: 1.0, 1: 8.0, 2: 9.0, 3: 2.0, 4: 7.0, 5: 10.0, 6: 3.0, 7: 5.0, 8: 0}
State-values for Policy 2: {0: 7.0, 1: 8.0, 2: 9.0, 3: 7.0, 4: 9.0, 5: 10.0, 6: 8.0, 7: 10.0, 8: 0}

EXAMPLE CALCULATION (Policy 1, State 2):
State 2 → Action down → State 5
Reward: -1
V(2) = -1 + 1.0 × V(5) = -1 + 1.0 × 10.0 = 9.0

POLICY COMPARISON:
Total value (Policy 1): 45.0
Total value (Policy 2): 68.0
Better policy: Policy 2


### 1.3 **State-Value Functions $V^\pi(s)$**

**Definition:**
- The **state-value function** $V^{\pi}(s)$ estimates how good it is to be in a given state $s$.
- The state-value function $V^{\pi}(s)$ represents the expected cumulative discounted reward starting from state $s$ and following policy $\pi$.

**Formal Definition**:
- $V^{\pi}(s) = \mathbb{E}_{\pi}[G_t | S_t = s]$
- Where:
    - $G_t = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$ is the discounted return
    - $\mathbb{E}_{\pi}$ denotes expectation under policy $\pi$
    - $\gamma$ is the discount factor (0 ≤ γ ≤ 1)
  
- Expanded Mathematical Form:
    - $V(s) = r_{s+1} + \gamma r_{s+2} + \gamma^2 r_{s+3} + \cdots + \gamma^{n-1} r_{s+n}$
    - $V^{\pi}(s) = \mathbb{E}_{\pi}[R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ... | S_t = s]$

- **Interpretation:**
    - $r_{s+1}$: Immediate reward after leaving state $s$
    - $\gamma r_{s+2}$: Next reward, discounted by factor $\gamma$
    - $\gamma^2 r_{s+3}$: Reward two steps later, discounted further
    - $\cdots$ Continues infinitely
    - $\gamma \in [0,1]$: Discount factor that balances **present vs. future rewards**

- **When to use:**
    - **Conceptual / Theoretical explanation** of value functions.
    - When introducing RL to beginners → easy to show “why future rewards are discounted.”
    - To **manually calculate returns** in very short episodes (e.g., toy problems like a 3-step grid world).
    - Useful in **Monte Carlo methods**, where we sample entire episodes and directly compute the return.
    - Not practical for real-world problems because we can’t compute `inf`

#### Grid World State-Values Example

In [ ]:
'''
- Computes how valuable each state is under a fixed action policy in a 3x3 grid world.
- Simulates following the policy from each state to accumulate rewards until reaching a terminal state or revisiting a state.
- Uses a simple deterministic transition function mapping an action and current position to a next position.
- Assigns rewards: -2 for mountain states, +10 for the terminal diamond state, and -1 for other states.
- Stops simulation to avoid infinite loops or once the terminal state is reached, adding its reward.
- Prints the total expected reward (state value) starting from each state when following the policy.
'''

def compute_state_value(state, policy, gamma=1.0):
    """Compute state value for deterministic policy"""
    if state == 8:  # Terminal state (diamond)
        return 0
    
    # Simulate following policy from this state
    current_state = state
    total_reward = 0
    visited = set()
    
    while current_state not in visited and current_state != 8:
        visited.add(current_state)
        action = policy[current_state]
        
        # Get reward for current state (simplified grid world)
        if current_state in [4, 7]:  # Mountain states
            reward = -2
        elif current_state == 8:     # Diamond state  
            reward = 10
        else:                       # Other states
            reward = -1
            
        total_reward += reward
        
        # Move to next state based on action (simplified transitions)
        next_state = get_next_state(current_state, action)
        current_state = next_state
    
    # Add terminal reward if reached
    if current_state == 8:
        total_reward += 10
        
    return total_reward


def get_next_state(state, action):
    """Simple deterministic transition function for 3x3 grid"""
    # Convert state to row, col
    row, col = state // 3, state % 3
    
    if action == 0:    # left
        col = max(0, col - 1)
    elif action == 1:  # down  
        row = min(2, row + 1)
    elif action == 2:  # right
        col = min(2, col + 1)
    elif action == 3:  # up
        row = max(0, row - 1)
    
    return row * 3 + col


# Define policy including terminal state with a dummy action (e.g. 0)
grid_policy = {0:1, 1:2, 2:1, 3:1, 4:3, 5:1, 6:2, 7:3, 8:0}
state_values = {}

for state in range(9):
    state_values[state] = compute_state_value(state, grid_policy)

print("State Values:")

for state, value in state_values.items():
    print(f"V({state}) = {value}")

State Values:
V(0) = 0
V(1) = 7
V(2) = 8
V(3) = 1
V(4) = 5
V(5) = 9
V(6) = 2
V(7) = 3
V(8) = 0


- **State-Value Interpretation:**
  
    - **High Values**: States that lead to goal with minimal cost
        - Closer to diamond with fewer mountain encounters
        - Optimal pathways through environment

    - **Low Values**: States requiring longer paths or mountain traversal
        - Farther from goal or poor intermediate positions
        - Suboptimal or risky pathways

    - **Zero Value**: Terminal goal state
      - No further rewards possible from terminal state
      - Standard convention in episodic tasks

### 1.4 **Computing State-Values with Recursion**

```python
def compute_state_value(state):  
    if state == terminal_state: 
        return 0 
 
    action = policy[state]  
    _, next_state, reward, _ = env.unwrapped.P[state][action][0]  
    return reward + gamma * compute_state_value(next_state)
```

In [ ]:
def compute_state_value_recursive(state, policy, env, gamma=1.0, memo=None, visited=None, max_depth=100):
    """
    Compute state value using recursive Bellman equation
    with memoization to handle cycles and recursion depth limit
    """
    if memo is None:
        memo = {}
        
    if visited is None:
        visited = set()

    # Base case: terminal state or max recursion depth reached
    if state == 15 or max_depth <= 0:  # terminal state or max depth
        return 0

    # Avoid cycles/infinite loops
    if state in visited:
        # Return 0 or memoized value to break cycle
        return memo.get(state, 0)

    # Check memo to avoid recomputation
    if state in memo:
        return memo[state]

    visited.add(state)

    # Get action from policy
    action = policy.get(state, 0)  # default action can be 0 if not in policy

    # For stochastic environments, compute expected value
    expected_value = 0
    transitions = env.unwrapped.P[state][action]

    for prob, next_state, reward, is_terminal in transitions:
        if is_terminal:
            value_contribution = reward
        else:
            value_contribution = reward + gamma * compute_state_value_recursive(
                next_state, policy, env, gamma, memo, visited, max_depth - 1)
        expected_value += prob * value_contribution

    visited.remove(state)
    memo[state] = expected_value
    return expected_value


# Example usage (assumes 'env' is the FrozenLake environment and 'frozen_lake_policy' is defined)

gamma = 1.0

frozen_lake_policy = {
    0: 1, 1: 2, 2: 1, 3: 1,    # First row
    4: 1, 5: 1, 6: 1, 7: 1,    # Second row  
    8: 2, 9: 1, 10: 1, 11: 1,  # Third row
    12: 2, 13: 2, 14: 2        # Fourth row (excluding terminal)
}

V = {}
for state in range(env.observation_space.n):
    V[state] = compute_state_value_recursive(state, frozen_lake_policy, env, gamma)

print("State Values for FrozenLake:")
for i in range(4):
    row = [f"V({j})={V.get(j, 0):.2f}" for j in range(i*4, (i+1)*4)]
    print(" | ".join(row))

State Values for FrozenLake:
V(0)=-3.00 | V(1)=8.00 | V(2)=9.00 | V(3)=-2.00
V(4)=-4.00 | V(5)=10.00 | V(6)=-1.00 | V(7)=-2.00
V(8)=10.00 | V(9)=0.00 | V(10)=0.00 | V(11)=0.00
V(12)=0.00 | V(13)=0.00 | V(14)=0.00 | V(15)=0.00


- **Code Explanation:**
    - **Memoization**: Prevents infinite recursion and improves efficiency
    - **Stochastic Handling**: Computes expected value over all possible transitions
    - **Terminal Handling**: Properly handles terminal states with zero continuation value
    - **Expected Value**: Weighted sum of outcomes by their probabilities

***

## **🔖2. The Bellman Equation**

### 2.1 **Bellman Equation (Recursive Form)**

- **Primary Concept:**
  - $V(s) = r_{s+1} + \gamma V(s+1)$
    - Where:
      - $r_{s+1}$ → Reward immediately after leaving state $s$
      - $\gamma V(s+1)$ → Discounted value of the next state, turning an infinite sum into a **recursive relationship**

- **State-Value Function Under Policy $\pi$:**
  - $V^{\pi}(s) = \sum_a \pi(a|s) \sum_{s'} P_{ss'}^a \left[ R_{ss'}^a + \gamma V^{\pi}(s') \right]$
  - For deterministic policies:
    - $V^{\pi}(s) = \sum_{s'} P_{ss'}^{\pi(s)} \left[ R_{ss'}^{\pi(s)} + \gamma V^{\pi}(s') \right]$
    - Where:
      - $\pi(s) $ = Action selected by deterministic policy in state $s$
      - $P_{ss'}^{\pi(s)}$ = Transition probability under policy action
      - $R_{ss'}^{\pi(s)}$ = Expected reward for the transition
      - $\gamma$ = Discount factor

### 2.2 **Mathematical Foundation of the Bellman Equation**

- **Primary Concept:**
  - The **Bellman Equation** is the cornerstone of dynamic programming in reinforcement learning, providing a recursive relationship between the value of a state and the values of its successor states.

- **Complete Mathematical Formulation:**

  - **For State-Value Functions:**
    - $V^{\pi}(s) = \sum_a \pi(a|s) \sum_{s'} P(s'|s,a) \left[ R(s,a,s') + \gamma V^{\pi}(s') \right]$

  - **For Deterministic Policies (simplified):**
    - $V^{\pi}(s) = \sum_{s'} P(s'|s,\pi(s)) \left[ R(s,\pi(s),s') + \gamma V^{\pi}(s') \right]$

  - **For Deterministic Environments (further simplified):**
    - $V^{\pi}(s) = R(s,\pi(s)) + \gamma V^{\pi}(s')$

- **Where:**
  - $V^{\pi}(s)$ = Value of state $s$ under policy $\pi$
  - $R(s,a,s')$ = Immediate reward for the transition from $s$ to $s'$ using action $$a$
  - $\gamma$ = Discount factor (0 ≤ $\gamma$ ≤ 1)
  - $P(s'|s,a)$ = Transition probability of moving to state $s'$ from $s$ by action $a$
  - $s'$ = Next state after taking action $a $ in state $s$

### 2.3 **Intuitive Understanding**

#### Why the Bellman Equation Works:

**Recursive Structure**: 
- The value of being in a state equals the immediate reward plus the (discounted) value of where you end up
- This creates a system of equations that can be solved for optimal values

**Optimality Principle**:
- Optimal solutions have the property that remaining decisions are optimal for the subproblem starting from the current state
- This enables breaking complex problems into simpler subproblems

**Dynamic Programming Foundation**:
- Enables bottom-up solution construction
- Avoids recomputing overlapping subproblems
- Guarantees convergence to optimal solution

### 2.4 **Bellman Equation Implementation**

In [ ]:
import numpy as np

def bellman_equation_single_state(state, policy, env, V, gamma=0.9):
    """
    Apply Bellman equation to compute value of a single state
    """
    if state >= env.observation_space.n - 1:  # Terminal state
        return 0
    
    action = policy[state]
    expected_value = 0
    
    # Get all possible transitions for this state-action pair
    transitions = env.unwrapped.P[state][action]
    
    for prob, next_state, reward, is_terminal in transitions:
        if is_terminal:
            # Terminal transition: only immediate reward
            value_contribution = reward
        else:
            # Non-terminal: immediate reward + discounted future value
            value_contribution = reward + gamma * V[next_state]
        
        expected_value += prob * value_contribution
    
    return expected_value

def bellman_update_all_states(policy, env, V, gamma=0.9):
    """
    Apply Bellman equation to update all state values
    """
    new_V = V.copy()
    
    for state in range(env.observation_space.n):
        if state < len(policy):  # Non-terminal state
            new_V[state] = bellman_equation_single_state(state, policy, env, V, gamma)
        else:
            new_V[state] = 0  # Terminal state
    
    return new_V

def iterative_policy_evaluation(policy, env, gamma=0.9, theta=1e-6, max_iterations=1000):
    """
    Iteratively apply Bellman equation until convergence
    """
    # Initialize value function
    V = np.zeros(env.observation_space.n)
    
    for iteration in range(max_iterations):
        new_V = bellman_update_all_states(policy, env, V, gamma)
        
        # Check for convergence
        delta = np.max(np.abs(new_V - V))
        V = new_V
        
        if delta < theta:
            print(f"Converged after {iteration + 1} iterations")
            break
    
    return V

# Example usage with FrozenLake
env = gym.make('FrozenLake-v1', is_slippery=True)

# Simple policy: always go right when possible, otherwise down
simple_policy = {
    0: 2, 1: 2, 2: 2, 3: 1,     # Row 1: right, right, right, down
    4: 2, 5: 1, 6: 1, 7: 1,     # Row 2: right, down, down, down  
    8: 2, 9: 2, 10: 1, 11: 1,   # Row 3: right, right, down, down
    12: 2, 13: 2, 14: 2         # Row 4: right, right, right
}

# Compute state values using Bellman equation
V_bellman = iterative_policy_evaluation(simple_policy, env, gamma=0.9)

print("State Values computed using Bellman Equation:")
for i in range(4):
    row = [f"{V_bellman[i*4 + j]:.3f}" for j in range(4)]
    print(" | ".join(f"{val:>6}" for val in row))

Converged after 30 iterations
State Values computed using Bellman Equation:
 0.015 |  0.015 |  0.034 |  0.015
 0.021 |  0.000 |  0.066 |  0.000
 0.054 |  0.159 |  0.219 |  0.000
 0.000 |  0.313 |  0.570 |  0.000


#### Code Explanation:
- **Single State Update**: Applies Bellman equation to compute new value for one state
- **Batch Update**: Updates all states using current value estimates
- **Iterative Process**: Repeats until values converge (change less than threshold)
- **Convergence Check**: Monitors maximum change in values across all states

### 2.5 **Bellman Optimality Equation**

#### Mathematical Formulation:
- **For Optimal State Values**:
  - $V^*(s) = \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma V^*(s')]$

- **For Optimal Action Values**:
  - $Q^*(s,a) = \sum_{s'} P(s'|s,a)[R(s,a,s') + \gamma \max_{a'} Q^*(s',a')]$

#### Key Differences from Policy-Specific Bellman Equation:
- **Optimization**: Takes maximum over actions instead of following fixed policy
- **Optimal Values**: Represents best possible performance from each state
- **Policy Independence**: Defines optimal values regardless of current policy

In [ ]:
def bellman_optimality_operator(V, env, gamma=0.9):
    """
    Apply Bellman optimality operator to find optimal values
    """
    new_V = np.zeros_like(V)
    
    for state in range(env.observation_space.n - 1):  # Exclude terminal state
        action_values = []
        
        for action in range(env.action_space.n):
            action_value = 0
            transitions = env.unwrapped.P[state][action]
            
            for prob, next_state, reward, is_terminal in transitions:
                if is_terminal:
                    action_value += prob * reward
                else:
                    action_value += prob * (reward + gamma * V[next_state])
            
            action_values.append(action_value)
        
        # Take maximum over all actions
        new_V[state] = max(action_values)
    
    return new_V

def value_iteration(env, gamma=0.9, theta=1e-6, max_iterations=1000):
    """
    Find optimal value function using value iteration
    """
    V = np.zeros(env.observation_space.n)
    
    for iteration in range(max_iterations):
        new_V = bellman_optimality_operator(V, env, gamma)
        
        delta = np.max(np.abs(new_V - V))
        V = new_V
        
        if delta < theta:
            print(f"Value iteration converged after {iteration + 1} iterations")
            break
    
    return V

# Find optimal values
V_optimal = value_iteration(env, gamma=0.9)
print("\nOptimal State Values:")
for i in range(4):
    row = [f"{V_optimal[i*4 + j]:.3f}" for j in range(4)]
    print(" | ".join(f"{val:>6}" for val in row))

Value iteration converged after 78 iterations

Optimal State Values:
 0.069 |  0.061 |  0.074 |  0.056
 0.092 |  0.000 |  0.112 |  0.000
 0.145 |  0.247 |  0.300 |  0.000
 0.000 |  0.380 |  0.639 |  0.000


### 2.6 **Properties and Convergence**

#### **Convergence Guarantees:**

- **Contraction Mapping**: Bellman operator is a contraction when γ < 1
    - Guaranteed convergence to unique fixed point
    - Convergence rate depends on discount factor

- **Fixed Point**: Optimal value function satisfies $V^* = T^*V^*$ where $T^*$ is Bellman optimality operator
    - Solution to system of Bellman optimality equations
    - Represents true optimal values

#### Computational Complexity:
- **Time Complexity**: $O(|S|²|A|)$ per iteration
    - Must examine all state-action pairs
    - Transition probabilities determine exact complexity

- **Space Complexity**: $O(|S|)$
    - Store value function for all states
    - Additional space for transition probabilities

- **Convergence Rate**: Geometric with rate γ
    - Faster convergence with smaller discount factors
    - Trade-off between solution quality and computation time

***

## **🔖3. Action-Value Functions $Q^\pi(s, a)$**

### 3.1 **what $Q^\pi(s,a)$ means**

* Formula:

  $Q^\pi(s,a) = \mathbb{E}_\pi\big[G_t \mid S_t=s, A_t=a\big]$

* Symbols:

  * $G_t$ = total return from time $t$: $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \dots$
  * $\mathbb{E}_\pi[\cdot]$ = expectation when actions after time $t$ are chosen according to policy $\pi$.
  * Condition $\mid S_t=s, A_t=a$ means: **we start in state $s$ and take action $a$ now**; randomness afterward is from environment transitions and the policy.
* Intuition: average (over possible randomness) of **immediate reward + discounted future rewards** after taking $a$ in $s$.
* Tiny numeric example:

  * Suppose $\gamma=0.9$, immediate reward $r=2$, and expected future return $V^\pi(s')=5$.
  * Then $Q^\pi(s,a)=2 + 0.9\times 5 = 6.5$.

### 3.2 **Break the return into immediate + future (derivation)**

* Start from definition:

  $Q^\pi(s,a) = \mathbb{E}\big[R_{t+1} + \gamma G_{t+1} \mid S_t=s, A_t=a\big]$

* Because expectation is linear:

  $Q^\pi(s,a) = \mathbb{E}[R_{t+1}\mid s,a] \;+\; \gamma \,\mathbb{E}[G_{t+1}\mid s,a]$
  
* Interpretations:

  * $\mathbb{E}[R_{t+1}\mid s,a]$ = expected immediate reward after doing $a$ in $s$.
  * $\mathbb{E}[G_{t+1}\mid s,a]$ = expected future return from the next time step onward.

### 3.3 **Deterministic one-step Bellman form**

* Formula (deterministic next state $s'$ and deterministic immediate reward $r_a$):
  
  $Q^\pi(s,a) = r_a + \gamma V^\pi(s')$

* Why this follows:
  * If taking $a$ from $s$ always lands in $s'$ and gives reward $r_a$, then $\mathbb{E}[R_{t+1}\mid s,a]=r_a$ and $\mathbb{E}[G_{t+1}\mid s,a]=V^\pi(s')$.
  
* Intuition: **immediate reward** + **discounted value of the known next state**.

### 3.4 **Stochastic transitions — expectation over next states**

* Formula:

  $Q^\pi(s,a) = \sum_{s'} P(s'\mid s,a)\,\big[ R(s,a,s') + \gamma V^\pi(s')\big]$

* Explanation:

  * If taking $a$ can lead to several possible next states $s'$, each with probability $P(s'|s,a)$, you average the immediate reward + future value for each possible $s'$.
* Numeric example (showing every arithmetic step):

  * Let two next states $s_1,s_2$ with probabilities $0.7$ and $0.3$.
  * Rewards: $R(s,a,s_1)=1$, $R(s,a,s_2)=2$.
  * Values: $V^\pi(s_1)=3$, $V^\pi(s_2)=4$.
  * $\gamma=0.9$.
  * Compute for $s_1$: inner = $1 + 0.9\times 3 = 2.7$.
    * Multiply by prob: $0.7\times 3.7 = 2.59$.
  
  * Compute for $s_2$: inner = $2 + 0.9\times 4 = 5.6$.
    * Multiply by prob: $0.3\times 5.6 = 1.68$.
    * So $Q^\pi(s,a) = 2.59 + 1.68 = 4.27$.

### 3.5 **Bellman expectation in terms of $Q$ (no $V$ needed)**

* Start: $V^\pi(s') = \sum_{a'} \pi(a'\mid s')\,Q^\pi(s',a')$.
  
* Substitute into previous expression:
  $Q^\pi(s,a) = \sum_{s'} P(s'\mid s,a)\Big[ R(s,a,s') + \gamma \sum_{a'} \pi(a'\mid s')\,Q^\pi(s',a')\Big]$

* Why useful:
  * This is a **self-consistent equation** for $Q^\pi$ only. Solve it (analytically or iteratively) to find the Q-values of a policy.

* Iteration form (policy evaluation):
  $Q_{k+1}(s,a) \leftarrow \sum_{s'} P(s'\mid s,a)\Big[R(s,a,s') + \gamma \sum_{a'} \pi(a'\mid s') Q_k(s',a')\Big]$

  * Keep updating $Q$ until it converges.

### 3.6 **Relationship $V^\pi$ ↔ $Q^\pi$**

* Formula:

  $V^\pi(s) = \sum_a \pi(a\mid s)\,Q^\pi(s,a)$

* Meaning: state-value = **expected Q-value when actions are sampled from $\pi$**.
* Quick numeric example:

  * Suppose two actions with probabilities $0.6$ and $0.4$.
  * $Q(s,a_1)=10,\; Q(s,a_2)=2$.
  * Then $V(s)=0.6\times 10 + 0.4\times 2 = 6.8$.

### 3.7 **Bellman *optimality* equation (why the $\max$ appears)**

* Formula:
  $Q^*(s,a) = \sum_{s'} P(s'\mid s,a)\Big[ R(s,a,s') + \gamma \max_{a'} Q^*(s',a')\Big]$

* Explanation:
  * $Q^*$ assumes **after taking action $a$ now we will act optimally thereafter**.
  * So the future value from $s'$ is the **maximum** Q-value over all actions available in $s'$: $V^*(s')=\max_{a'}Q^*(s',a')$

* Small numeric illustration:
  * Suppose deterministic next state $s'$, $R=1$, $\gamma=0.9$, and $\max_{a'}Q^*(s',a')=9$
  * Then $Q^*(s,a)=1 + 0.9\times 9 = 9.1$.

* The optimal policy uses:
  $\pi^*(s) = \arg\max_a Q^*(s,a)$

  * Pick the action that yields the largest $Q^*$ in that state.

### 3.8 **Sample-based updates (how algorithms use these formulas)**

* **Q-Learning (off-policy)** one-step sample update (common in practice):

  $Q(s,a) \leftarrow Q(s,a) + \alpha\big[ r + \gamma\max_{a'}Q(s',a') - Q(s,a)\big]$

  * Use when you sample a transition $(s,a,r,s')$.
  * Intuition: move $Q(s,a)$ toward the sample target $r + \gamma\max_{a'}Q(s',a')$.

* **SARSA (on-policy)**:
  $Q(s,a) \leftarrow Q(s,a) + \alpha\big[ r + \gamma Q(s',a') - Q(s,a)\big]$

  * Here $a'$ is the actual next action chosen by current policy; used when learning while following that policy.

### 3.9 **Terminal-state conventions**

* Common options:
  * Set $Q(\text{terminal},a)=0$ for all $a$.
  * Or skip/ignore those states in updates.
  
* Both are fine if applied consistently.

### 3.10 **Quick intuitive checklist** 🧭

* If you *know* the next state exactly → use $Q = r + \gamma V(\text{next})$.
* If next state is random → average over next states with $P(s'|s,a)$.
* If you have a policy $\pi$ and want to compute its values → use the Bellman expectation (in terms of $Q$ or $V$).
* If you want the best possible behavior → replace expected future actions by $\max$ (Bellman optimality), then act greedily w\.r.t. $Q^*$.

### Complete Q-Value Computation

```python
def compute_q_value(state, action):  
    if state == terminal_state: 
        return None 
 
    _, next_state, reward, _ = env.unwrapped.P[state][action][0] 
    return reward + gamma * compute_state_value(next_state)
```

In [ ]:
# ============================================
# 1. Import Required Libraries
# ============================================
import gymnasium as gym
import numpy as np


# ============================================
# 2. Initialize Environment & Parameters
# ============================================
env = gym.make('FrozenLake-v1', is_slippery=True)

num_states = env.observation_space.n     # Number of states in FrozenLake
num_actions = env.action_space.n         # Number of possible actions
terminal_state = 15                      # Goal state index in FrozenLake
gamma = 0.9                              # Discount factor


# ============================================
# 3. Initialize Value Function
# ============================================
# Value function stores the "goodness" of each state
V = np.zeros(num_states)     # Start with all states = 0
V[terminal_state] = 1.0      # Goal state is assigned value 1


# ============================================
# 4. Q-Value Computation Function
# ============================================
def compute_q_value(state, action, V):
    """
    Compute the Q-value for a given (state, action) pair.

    Parameters:
        state  (int): Current state
        action (int): Action taken in this state
        V      (array): Current value function

    Returns:
        float: Estimated Q-value
    """
    # Terminal state has no future rewards
    if state == terminal_state:
        return 0
    
    # env.unwrapped.P[state][action] gives the transition dynamics:
    # [(probability, next_state, reward, done), ...]
    probability, next_state, reward, done = env.unwrapped.P[state][action][0]
    
    # Bellman expectation equation
    return reward + gamma * V[next_state]


# ============================================
# 5. Compute Q-values for All State-Action Pairs
# ============================================
Q = {
    (state, action): compute_q_value(state, action, V)
    for state in range(num_states)
    for action in range(num_actions)
}

print("Q-values:")
print(Q)


# ============================================
# 6. Greedy Policy Improvement
# ============================================
def improve_policy(Q, num_states, num_actions):
    """
    Improve policy using greedy selection:
    For each state, pick the action with the maximum Q-value.
    """
    improved_policy = {}
    
    for state in range(num_states - 1):  # Exclude terminal state
        max_action = max(range(num_actions), key=lambda action: Q[(state, action)])
        improved_policy[state] = max_action
    
    return improved_policy


# Generate improved policy
policy = improve_policy(Q, num_states, num_actions)

print("\nImproved Policy:")
print(policy)


# ============================================
# 7. Test the Improved Policy
# ============================================
state, _ = env.reset()   # Reset environment to initial state
terminated = False

while not terminated:
    action = policy[state]  # Select action according to policy
    state, reward, terminated, truncated, info = env.step(action)
    print(f"State: {state}, Reward: {reward}")

env.close()

Q-values:
{(0, 0): np.float64(0.0), (0, 1): np.float64(0.0), (0, 2): np.float64(0.0), (0, 3): np.float64(0.0), (1, 0): np.float64(0.0), (1, 1): np.float64(0.0), (1, 2): np.float64(0.0), (1, 3): np.float64(0.0), (2, 0): np.float64(0.0), (2, 1): np.float64(0.0), (2, 2): np.float64(0.0), (2, 3): np.float64(0.0), (3, 0): np.float64(0.0), (3, 1): np.float64(0.0), (3, 2): np.float64(0.0), (3, 3): np.float64(0.0), (4, 0): np.float64(0.0), (4, 1): np.float64(0.0), (4, 2): np.float64(0.0), (4, 3): np.float64(0.0), (5, 0): np.float64(0.0), (5, 1): np.float64(0.0), (5, 2): np.float64(0.0), (5, 3): np.float64(0.0), (6, 0): np.float64(0.0), (6, 1): np.float64(0.0), (6, 2): np.float64(0.0), (6, 3): np.float64(0.0), (7, 0): np.float64(0.0), (7, 1): np.float64(0.0), (7, 2): np.float64(0.0), (7, 3): np.float64(0.0), (8, 0): np.float64(0.0), (8, 1): np.float64(0.0), (8, 2): np.float64(0.0), (8, 3): np.float64(0.0), (9, 0): np.float64(0.0), (9, 1): np.float64(0.0), (9, 2): np.float64(0.0), (9, 3): np.flo

In [ ]:
def compute_q_value_deterministic(state, action, env, V, gamma=1.0):
    """
    Compute Q-value for deterministic environment
    """
    if state >= env.observation_space.n - 1:  # Terminal state
        return 0
    
    # For deterministic transition, take first (and only) outcome
    transitions = env.unwrapped.P[state][action]
    prob, next_state, reward, is_terminal = transitions[0]
    
    if is_terminal:
        return reward
    else:
        return reward + gamma * V[next_state]

def compute_q_value_stochastic(state, action, env, V, gamma=1.0):
    """
    Compute Q-value for stochastic environment (expected value)
    """
    if state >= env.observation_space.n - 1:  # Terminal state
        return 0
    
    expected_q_value = 0
    transitions = env.unwrapped.P[state][action]
    
    for prob, next_state, reward, is_terminal in transitions:
        if is_terminal:
            contribution = reward
        else:
            contribution = reward + gamma * V[next_state]
        
        expected_q_value += prob * contribution
    
    return expected_q_value

# Example: Computing all Q-values for a specific state
def compute_all_q_values_for_state(state, env, V, gamma=1.0):
    """
    Compute Q-values for all actions in a given state
    """
    q_values = {}
    action_names = ['Left', 'Down', 'Right', 'Up']
    
    for action in range(env.action_space.n):
        q_val = compute_q_value_stochastic(state, action, env, V, gamma)
        q_values[action] = q_val
        print(f"Q({state}, {action_names[action]}) = {q_val:.3f}")
    
    return q_values

# Example state values (from previous computation)
example_V = {0: 1,
            1: 8,
            2: 9, 
            3: 2, 
            4: 7, 
            5: 10, 
            6: 3, 
            7: 5, 
            8: 0}

# Compute Q-values for state 4
print("Q-values for State 4:")

q_values_state_4 = compute_all_q_values_for_state(4, env, example_V, gamma=1.0)

Q-values for State 4:
Q(4, Left) = 2.667
Q(4, Down) = 2.333
Q(4, Right) = 0.333
Q(4, Up) = 2.667


In [ ]:
def compute_all_q_values(policy, env, V, gamma=0.9):
    """
    Compute Q-values for all state-action pairs
    """
    Q = {}
    
    for state in range(env.observation_space.n):
        for action in range(env.action_space.n):
            if state >= env.observation_space.n - 1:  # Terminal state
                Q[(state, action)] = None
            else:
                Q[(state, action)] = compute_q_value_stochastic(state, action, env, V, gamma)
    return Q

def display_q_table(Q, env):
    """
    Display Q-values in a readable table format
    """
    action_names = ['Left', 'Down', 'Right', 'Up']
    
    print("\nComplete Q-Value Table:")
    print("=" * 60)
    print(f"{'State':<6} {'Left':<8} {'Down':<8} {'Right':<8} {'Up':<8}")
    print("-" * 60)
    
    for state in range(env.observation_space.n - 1):  # Exclude terminal
        row = [f"{state:<6}"]
        for action in range(env.action_space.n):
            q_val = Q[(state, action)]
            if q_val is not None:
                row.append(f"{q_val:<8.3f}")
            else:
                row.append(f"{'N/A':<8}")
        print(" ".join(row))

# Compute complete Q-table
complete_Q = compute_all_q_values(simple_policy, env, V_bellman, gamma=0.9)
display_q_table(complete_Q, env)


Complete Q-Value Table:
State  Left     Down     Right    Up      
------------------------------------------------------------
0      0.015    0.015    0.015    0.014   
1      0.009    0.015    0.015    0.019   
2      0.034    0.029    0.034    0.019   
3      0.015    0.015    0.009    0.019   
4      0.027    0.022    0.021    0.011   
5      0.000    0.000    0.000    0.000   
6      0.076    0.066    0.076    0.010   
7      0.000    0.000    0.000    0.000   
8      0.022    0.064    0.054    0.070   
9      0.110    0.176    0.159    0.082   
10     0.239    0.219    0.191    0.068   
11     0.000    0.000    0.000    0.000   
12     0.000    0.000    0.000    0.000   
13     0.142    0.265    0.313    0.219   
14     0.330    0.598    0.570    0.493   


### 3.11 **Quick Comparison: $V^\pi$ Vs $Q^\pi$**

| Function         | Input                 | Output         | Question it answers                                         | Intuition                                                                                              | Example                                                                                         |
| ---------------- | --------------------- | -------------- | ----------------------------------------------------------- | ------------------------------------------------------------------------------------------------------ | ----------------------------------------------------------------------------------------------- |
| **$V^\pi(s)$**   | State $s$             | Number (value) | “How good is this state under policy $\pi$?”                | Expected long-term return starting **from state $s$** and following policy $\pi$.                      | In chess: value of a board position assuming you continue playing according to strategy $\pi$.  |
| **$Q^\pi(s,a)$** | State $s$, Action $a$ | Number (value) | “How good is this action in this state under policy $\pi$?” | Expected long-term return starting **from state $s$**, taking action $a$, then following policy $\pi$. | In chess: value of moving a knight to a specific square (action) given the current board state. |



### 🔑 Key Differences

1. **Scope**

   * $V^\pi(s)$ looks at **how desirable a state is overall**.
   * $Q^\pi(s,a)$ looks at **how desirable a specific action is in that state**.

2. **Decision-making granularity**

   * $V^\pi(s)$ is coarse-grained (state-level).
   * $Q^\pi(s,a)$ is fine-grained (state-action level, more detailed).

3. **Relation**

   * $V^\pi(s) = \sum_a \pi(a|s)\, Q^\pi(s,a)$
     (state value is the expected action value under the policy).

4. **Use cases**

   * $V^\pi(s)$: Used in **policy evaluation** (how good is my strategy overall?).
   * $Q^\pi(s,a)$: Used in **policy improvement** and **control** (which action should I pick?).



### ⭐ Intuitive Summary

* **$V$** = *“Value of being in a place (state).”*
* **$Q$** = *“Quality of making a move (action) in that place.”*